In [39]:
import requests, math
from skyfield.api import load, wgs84, EarthSatellite
from skyfield.iokit import parse_tle_file
import numpy as np
import plotly.graph_objects as go

In [4]:
# import tle data
# for group can be used e.g.: active, stations, cubesat
def importSatellitesFromCelestrak(group):
    # URL to fetch tle data
    url = f"https://celestrak.org/NORAD/elements/gp.php?GROUP={group}&FORMAT=tle"

    try:
        # Sending a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Get the content as a string
        content = response.text
        # Clean up
        content = content.strip()
        # Split the content into array
        content = content.splitlines()

        satellites = {}
        key = 0

        # Create JSON object
        for i, line in enumerate(content):
            if i % 3 == 0:  # Every 3rd line: Satellite name
                name = line.strip()
            elif i % 3 == 1:  # Every 3rd line + 1: Line 1
                line1 = line
                sat_id = line.split(' ')[1].replace('U', '')
            elif i % 3 == 2:  # Every 3rd line + 2: Line 2 and complete the entry
                line2 = line
                satellites[key] = {
                    'name': name,
                    'id': sat_id,
                    'object': [line1, line2]
                }
                key += 1


    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

    # return tle data as JSON object
    return satellites

satellites = importSatellitesFromCelestrak('stations')
print(satellites)

{0: {'name': 'ISS (ZARYA)', 'id': '25544', 'object': ['1 25544U 98067A   25058.66013045  .00061406  00000+0  10908-2 0  9991', '2 25544  51.6385 129.5122 0005976 320.1405 159.8209 15.49585351498173']}, 1: {'name': 'CSS (TIANHE)', 'id': '48274', 'object': ['1 48274U 21035A   25058.16152894  .00052524  00000+0  61616-3 0  9992', '2 48274  41.4670 330.6361 0007195 252.7606 107.2446 15.60608211218950']}, 2: {'name': 'ISS (NAUKA)', 'id': '49044', 'object': ['1 49044U 21066A   25058.50966911  .00028468  00000+0  51274-3 0  9992', '2 49044  51.6359 130.2610 0006124 322.7999  37.2565 15.49551992203170']}, 3: {'name': 'FREGAT DEB', 'id': '49271', 'object': ['1 49271U 11037PF  25057.49912949  .00013245  00000+0  28388-1 0  9995', '2 49271  51.6517 331.0958 0918907 312.5221  40.1464 12.27672528166306']}, 4: {'name': 'CSS (WENTIAN)', 'id': '53239', 'object': ['1 53239U 22085A   25058.16152894  .00052524  00000+0  61616-3 0  9995', '2 53239  41.4670 330.6361 0007195 252.7606 107.2446 15.60608211214

In [18]:
# get track data of the satellite for one revolution
# returns object with timestamps describing rise, culminate, set, complete timeline
# latitude, longitude, altitude, azimuth, distance and range_rate

# dateString: String Format YYYY-MM-DDTHH:MM
def convertDateStringToArray(dateString):

    date_part, time_part = dateString.split("T")

    # Split further into components and convert to integers
    year, month, day = map(int, date_part.split("-"))
    hour, minute = map(int, time_part.split(":"))

    # Combine into a list
    datetime_array = [year, month, day, hour, minute]

    return np.array(datetime_array)

# satelliteObject: taken from satellites e.g satellites[0]
# location: Array [String location name, Float latitude, Float longitude]
# time: Array [Array start [Int year, month, day, hour, min], Array end[...]]
# elevation: Float
def getTrackDataRevolution(satelliteObject, location, time, elevation):

    try:
        line1 = satelliteObject[0]
        line2 = satelliteObject[1]
        satellite = EarthSatellite(line1, line2)
        latitude, longitude = location[1], location[2]
        starttime, stoptime = time[0], time[1]

        ts = load.timescale()

        bluffton = wgs84.latlon(latitude, longitude)

        # for Satellite altitude, azimuth, and distance¶
        difference = satellite - bluffton

        # for 'rise', 'culminate', 'set'
        t0 = ts.utc(starttime[0], starttime[1], starttime[2], starttime[3], starttime[4])
        t1 = ts.utc(stoptime[0], stoptime[1], stoptime[2], stoptime[3], stoptime[4])

        data = {}
        t, events = satellite.find_events(bluffton, t0, t1, altitude_degrees=elevation)
        event_names = 'rise', 'culminate', 'set'
        i = 0
        value = []

        for ti, event in zip(t, events):
            name = event_names[event]
            value.append(ti.utc_strftime("%Y-%m-%dT%H:%M"))
            if name == 'set':
                data[i] = {
                    'rise': value[0], 
                    'culminate': value[1], 
                    'set': value[2]
                }
                i += 1
                value = []

        for i in range(len(data)):
            starttime = convertDateStringToArray(data[i]['rise'])
            stoptime = convertDateStringToArray(data[i]['set'])

            timeline = ts.utc(starttime[0], starttime[1], starttime[2], starttime[3], np.arange(0,100,1))
            data[i]['timeline'] = list(timeline.utc_strftime('%Y-%m-%dT%H:%M'))

            geocentric = satellite.at(timeline) 
            subpoint = geocentric.subpoint() # find sub sat point 
            
            lat, lon = wgs84.latlon_of(geocentric)

            data[i]['lat'] = list(lat.degrees)
            data[i]['lon'] = list(lon.degrees)

            topocentric = difference.at(timeline)
            alt, az, distance = topocentric.altaz()

            data[i]['alt'] = list(alt.degrees)
            data[i]['az'] = list(az.degrees)
            data[i]['distance'] = list(distance.km)

            pos = (satellite - bluffton).at(timeline)
            _, _, range_val, _, _, range_rate = pos.frame_latlon_and_rates(bluffton)
            
            data[i]['range'] = list(range_val.km)
            data[i]['range_rate'] = list(range_rate.km_per_s)
            
        return data
    
    except:
        
        return {'error': 'unable to create data'}

satelliteObject = ['1 39444U 13066AE  25058.56821037  .00022544  00000+0  17951-2 0  9993', '2 39444  97.7531  20.4205 0042284  13.3448 346.8882 14.99667078608284']
location = ['Bremen', 53.0793, 8.8017]
time = [[2025, 2, 27, 22, 40], [2025, 2, 28, 22, 40]]
elevation = 0

trackDataRevolution = getTrackDataRevolution(satelliteObject, location, time, elevation)
print(trackDataRevolution)


{0: {'rise': '2025-02-28T01:20', 'culminate': '2025-02-28T01:25', 'set': '2025-02-28T01:30', 'timeline': ['2025-02-28T01:00', '2025-02-28T01:01', '2025-02-28T01:02', '2025-02-28T01:03', '2025-02-28T01:04', '2025-02-28T01:05', '2025-02-28T01:06', '2025-02-28T01:07', '2025-02-28T01:08', '2025-02-28T01:09', '2025-02-28T01:10', '2025-02-28T01:11', '2025-02-28T01:12', '2025-02-28T01:13', '2025-02-28T01:14', '2025-02-28T01:15', '2025-02-28T01:16', '2025-02-28T01:17', '2025-02-28T01:18', '2025-02-28T01:19', '2025-02-28T01:20', '2025-02-28T01:21', '2025-02-28T01:22', '2025-02-28T01:23', '2025-02-28T01:24', '2025-02-28T01:25', '2025-02-28T01:26', '2025-02-28T01:27', '2025-02-28T01:28', '2025-02-28T01:29', '2025-02-28T01:30', '2025-02-28T01:31', '2025-02-28T01:32', '2025-02-28T01:33', '2025-02-28T01:34', '2025-02-28T01:35', '2025-02-28T01:36', '2025-02-28T01:37', '2025-02-28T01:38', '2025-02-28T01:39', '2025-02-28T01:40', '2025-02-28T01:41', '2025-02-28T01:42', '2025-02-28T01:43', '2025-02-28T01

In [14]:
# get track data of the satellite for within given elevation
# returns object with timestamps describing rise, culminate, set, complete timeline
# latitude, longitude, altitude, azimuth, distance and range_rate
def getTrackDataElevation(trackData, elevation):
        
    data = {}
    for i in range(len(trackData)):
        row = trackData[i]
        timeline, lat, lon, alt, az, distance, range_rate = [],[],[],[],[],[],[] 
        for j in range(len(row['alt'])):
            
            if row['alt'][j] >= elevation:

                timeline.append(row['timeline'][j])
                lat.append(row['lat'][j])
                lon.append(row['lon'][j])
                alt.append(row['alt'][j])
                az.append(row['az'][j])
                distance.append(row['distance'][j])
                range_rate.append(row['range_rate'][j])
                
        
        if len(timeline) > 0:
            data[i] = {
                'timeline': timeline,
                'lat': lat,
                'lon': lon,
                'alt': alt,
                'az': az,
                'distance': distance,
                'range_rate': range_rate
            }
        
    return data

trackDataElevation = getTrackDataElevation(trackDataRevolution, elevation)
print(trackDataElevation)

{0: {'timeline': ['2025-02-28T01:20', '2025-02-28T01:21', '2025-02-28T01:22', '2025-02-28T01:23', '2025-02-28T01:24', '2025-02-28T01:25', '2025-02-28T01:26', '2025-02-28T01:27', '2025-02-28T01:28', '2025-02-28T01:29'], 'lat': [68.52824243421081, 65.0204237388127, 61.45884773372338, 57.86119770383267, 54.23813433059504, 50.59642858369095, 46.94057976231187, 43.27370241274019, 39.59803783872153, 35.91526206325265], 'lon': [42.91953269878281, 39.43063453609158, 36.69978669650753, 34.48049636287879, 32.620397639617586, 31.020874405529778, 29.615502880363024, 28.35797001687939, 27.214993346479, 26.162002680273986], 'alt': [1.4773785943807476, 4.612223626748021, 7.7623949220602375, 10.6040729013733, 12.544546667605188, 12.941116834263358, 11.637950861444331, 9.129058054744048, 6.078907623849504, 2.9281270574801965], 'az': [32.991011580165555, 40.30924283250681, 49.66431305840153, 61.494121540323434, 75.74217054140026, 91.32162520217439, 106.32551248435736, 119.21766621427251, 129.54990796558

In [33]:
# create groundtrack plot
# visualising complete revolution as red
# and green within elevation
def createPlotGroundTrack(trackData, trackDataElevation, location):
    latitude, longitude = location[1], location[2]

    # Create the map
    fig = go.Figure()

    # Add a scatter point for the location
    fig.add_trace(go.Scattergeo(
        lon=[longitude],
        lat=[latitude],
        mode='markers+text',
        text=[location[0]],
        textposition="top right",
        marker=dict(size=8, color="blue"),
        name="Location"
    ))

    # Add the ground track data
    fig.add_trace(go.Scattergeo(
        lon=trackData['lon'],
        lat=trackData['lat'],
        text=[timeline.replace('T', ' ') for timeline in trackData['timeline']],
        mode='lines+markers+text',
        line=dict(color="red"),
        textfont=dict(color="rgba(0, 0, 0, 0)"),
        marker=dict(size=4),
        name="Ground Track"
    ))

    # Add the ground track data for minimum elevation
    fig.add_trace(go.Scattergeo(
        lon=trackDataElevation['lon'],
        lat=trackDataElevation['lat'],
        text=[timeline.replace('T', ' ') for timeline in trackDataElevation['timeline']],
        mode='lines+markers+text',
        line=dict(color="green"),
        textfont=dict(color="rgba(0, 0, 0, 0)"),
        marker=dict(size=4),
        name="Min Elevation Track"
    ))

    # Customize map appearance
    fig.update_geos(
        showcoastlines=True,
        coastlinecolor="Black",
        showland=True,
        landcolor="lightgray",
        showocean=True,
        oceancolor="lightblue",
        projection_type="natural earth"
    )

    # Add a title and layout
    fig.update_layout(
        title="Ground Track Visualization",
        geo=dict(
            showframe=False,
            showlakes=True,
            lakecolor="blue"
        ),
        margin={"r": 0, "t": 30, "l": 0, "b": 0}
    )

    return fig


fig = createPlotGroundTrack(trackDataRevolution[0], trackDataElevation[0], location)
fig.show()

In [35]:
# create polar plot
def createPlotPolar(trackDataElevation):
    fig = go.Figure()

    # Convert azimuth to radians and adjust altitude for polar representation
    theta = [az * np.pi / 180 for az in trackDataElevation['az']]
    r = [90 - alt for alt in trackDataElevation['alt']]

    fig.add_trace(go.Scatterpolar(
        r=r,
        theta=trackDataElevation['az'],
        mode='lines+markers',
        name="Polar Track",
        marker=dict(color="green")
    ))

    fig.update_layout(
        polar=dict(
            angularaxis=dict(rotation=-90),
            radialaxis=dict(range=[0, 90], tickvals=[30, 60, 90])
        ),
        title="Polar Plot (Altitude vs Azimuth)"
    )
    return fig

fig = createPlotPolar(trackDataElevation[0])
fig.show()


In [36]:
# create Altitude and Azimuth plot

def createPlotAltAzi(trackDataElevation):
    fig = go.Figure()

    # Plot Altitude
    fig.add_trace(go.Scatter(
        x=trackDataElevation['timeline'],
        y=trackDataElevation['alt'],
        mode='lines',
        name="Altitude",
        line=dict(color="blue")
    ))

   # Plot Azimuth
    fig.add_trace(go.Scatter(
        x=trackDataElevation['timeline'],
        y=trackDataElevation['az'],
        mode='lines',
        name="Azimuth",
        line=dict(color="red")
    ))

    # Update layout with correctly placed grid settings
    fig.update_layout(
        title="Altitude and Azimuth over Time",
        xaxis=dict(
            title="Date and Time",
            showgrid=True  # Enable gridlines for the x-axis
        ),
        yaxis=dict(
            title="Angles (°)",
            showgrid=True  # Enable gridlines for the y-axis
        ),
        legend=dict(
            title="Legend"
        )
    )

    return fig

fig = createPlotAltAzi(trackDataElevation[0])
fig.show()

In [38]:
# create Distance plot

def createPlotDistance(trackDataElevation):
    fig = go.Figure()

    # Plot Distance
    fig.add_trace(go.Scatter(
        x=trackDataElevation['timeline'],
        y=trackDataElevation['distance'],
        mode='lines',
        name="Distance",
        line=dict(color="green")
    ))

    # Plot Speed
    fig.add_trace(go.Scatter(
        x=trackDataElevation['timeline'],
        y=trackDataElevation['range_rate'],  # Speed in km/s
        mode='lines',
        name="range’s rate of change",
        line=dict(color="blue"),
        yaxis="y2"  # Associate this trace with the second y-axis
    ))

    # Update layout for dual y-axes
    fig.update_layout(
        title="Distance and Rate of Change over Time",
        xaxis=dict(
            title="Date and Time",
            showgrid=True  # Enable gridlines for the x-axis
        ),
        yaxis=dict(
            title="Distance (km)",
            showgrid=True,  # Enable gridlines for the primary y-axis
            side="left"     # Place the primary y-axis on the left
        ),
        yaxis2=dict(
            title="rate of change (km/s)",
            overlaying="y",  # Overlay on the same plot
            side="right",    # Place the secondary y-axis on the right
            showgrid=False   # Optional: Disable gridlines for clarity
        ),
        legend=dict(
            title="Legend"
        )
    )

    return fig

fig = createPlotDistance(trackDataElevation[0])
fig.show()

In [40]:
# Create LinkBudget

def getLinkBudget(bandwidth, eirp, distance, frequency, antennaGainRx_dB, receiverGain_dB, bitrate, bitsPerSymbol, rollOffFactor, noiseFigure_dB, txPower, noiseTempAn):

    antennaGainRx = 10**(antennaGainRx_dB/10)
    receiverGain = 10**(receiverGain_dB/10)
    noiseFigure = 10**(noiseFigure_dB/10)

    k = 1.38e-23
    temp_0 = 290
    wavelength = 3e8 / frequency

    antennaGainTx = eirp / txPower
    antennaGainTx_dB = 10*np.log10(antennaGainTx)
    pathloss = ((4 * np.pi * distance) / wavelength)**2
    pathloss_dB =  10*np.log10(pathloss)
    powerFluxDensity = eirp / (4 * np.pi * distance**2)

    noiseTempEq = (noiseFigure - 1) * temp_0
    noiseOut = k * receiverGain * (noiseTempAn + noiseTempEq) * bandwidth
    noiseOut_dB = 10*np.log10(noiseOut)
    noiseIn = k * noiseTempAn * bandwidth
    noiseIn_dB = 10*np.log10(noiseIn)

    signalIn = antennaGainTx * txPower * antennaGainRx * 1/pathloss
    signalIn_dB = 10*np.log10(signalIn)

    signalOut = signalIn * receiverGain
    signalOut_dB = 10*np.log10(signalOut)

    snrIn = signalIn / noiseIn
    snrIn_dB =  10*np.log10(snrIn)
    snrOut = signalOut / noiseOut
    snrOut_dB =  10*np.log10(snrOut)
    
    # Wert zu hoch -> logarithmisch anderer Wert
    attentuation = txPower / signalIn

    antennaEffectiveArea = antennaGainRx * wavelength**2 / (4*np.pi)
    antennaEfficiency = 0.6
    antennaArea = antennaEffectiveArea / antennaEfficiency
    antennaDiameter = 2* np.sqrt(antennaArea/np.pi)

    M = 2**bitsPerSymbol

    symbolRate = bitrate/bitsPerSymbol
    spectralEfficiency = np.log2(M)/(1+rollOffFactor)

    requiredBandwidth = bitrate / spectralEfficiency

    energyPerBit = snrOut * requiredBandwidth / bitrate

    bitErrorRatio = 1/2 * math.erfc(np.sqrt(energyPerBit))

    txPower_dB = 10*np.log10(txPower)
    eirp_dB = 10*np.log10(eirp)
    bandwidth_dB = 10*np.log10(bandwidth)

    govert_dB = 10*np.log10(antennaGainRx / (noiseTempAn+noiseTempEq))

    result = {
        'frequency': {'value': frequency, 'dB': "", 'unit': ["Hz", ""]},
        'wavelength': {'value': wavelength, 'dB': "", 'unit': ["m",""]},
        'bandwidth': {'value': bandwidth, 'dB': bandwidth_dB, 'unit': ["Hz", "dB"]},
        'txPower': {'value': txPower, 'dB': txPower_dB, 'unit': ["W", "dBW"]},
        'antennaGainTx': {'value': antennaGainTx, 'dB': antennaGainTx_dB, 'unit': ["", "dBi"]},
        'eirp': {'value': eirp, 'dB': eirp_dB, 'unit': ["W", "dBW"]},
        'distance': {'value': distance, 'dB': "", 'unit': ["m", ""]},
        'pathloss': {'value': pathloss, 'dB': pathloss_dB, 'unit': ["", "dB"]},
        'powerFluxDensity': {'value': powerFluxDensity, 'dB': "", 'unit': ["W/m²", ""]},
        'antennaDiameter': {'value': antennaDiameter, 'dB': "", 'unit': ["m", ""]},
        'antennaArea': {'value': antennaArea, 'dB': "", 'unit': ["m²", ""]},
        'antennaEffectiveArea': {'value': antennaEffectiveArea, 'dB': "", 'unit': ["m²", ""]},
        'antennaEfficiency': {'value': antennaEfficiency * 100, 'dB': "", 'unit': ["%", ""]},
        'antennaGainRx': {'value': antennaGainRx, 'dB': antennaGainRx_dB, 'unit': ["", "dBi"]},
        'signalIn': {'value': signalIn, 'dB': signalIn_dB, 'unit': ["W", "dBW"]},
        'receiverGain': {'value': receiverGain, 'dB': receiverGain_dB, 'unit': ["", "dB"]},
        'signalOut': {'value': signalOut, 'dB': signalOut_dB, 'unit': ["W", "dBW"]},
        'noiseTempAn': {'value': noiseTempAn, 'dB': "", 'unit': ["K", ""]},
        'noiseIn': {'value': noiseIn, 'dB': noiseIn_dB, 'unit': ["W", "dBW"]},
        'snrIn': {'value': snrIn, 'dB': snrIn_dB, 'unit': ["", "dB"]},
        'noiseFigure': {'value': noiseFigure, 'dB': noiseFigure_dB, 'unit': ["", "dB"]},
        'noiseTempEq': {'value': noiseTempEq, 'dB': "", 'unit': ["K", ""]},
        'noiseOut': {'value': noiseOut, 'dB': noiseOut_dB, 'unit': ["W", "dBW"]},
        'snrOut': {'value': snrOut, 'dB': snrOut_dB, 'unit': ["", "dB"]},
        'bitsPerSymbol': {'value': bitsPerSymbol, 'dB': "", 'unit': ["", ""]},
        'bitrate': {'value': bitrate, 'dB': "", 'unit': ["bits/s", ""]},
        'symbolRate': {'value': symbolRate, 'dB': "", 'unit': ["Sym/s", ""]},
        'rollOffFactor': {'value': rollOffFactor, 'dB': "", 'unit': ["", ""]},
        'requiredBandwidth': {'value': requiredBandwidth, 'dB': "", 'unit': ["Hz", ""]},
        'energyPerBit': {'value': energyPerBit, 'dB': "", 'unit': ["", ""]},
        'bitErrorRatio': {'value': bitErrorRatio, 'dB': "", 'unit': ["", ""]},
        'govert': {'value': "", 'dB': govert_dB, 'unit': ["", "dB/K"]}
    }

    return result

bandwidth = 1000000
eirp = 2
distance = 804437
frequency = 145.950e6
antennaGainRx_dB = 18
receiverGain_dB = 60
bitrate = 1200
bitsPerSymbol = 1
rollOffFactor = 0.3
noiseFigure_dB = 2
txPower = 1
noiseTempAn = 1000

linkbudetData = getLinkBudget(bandwidth, eirp, distance, frequency, antennaGainRx_dB, receiverGain_dB, bitrate, bitsPerSymbol, rollOffFactor, noiseFigure_dB, txPower, noiseTempAn)
print(linkbudetData)

{'frequency': {'value': 145950000.0, 'dB': '', 'unit': ['Hz', '']}, 'wavelength': {'value': 2.055498458376156, 'dB': '', 'unit': ['m', '']}, 'bandwidth': {'value': 1000000, 'dB': 60.0, 'unit': ['Hz', 'dB']}, 'txPower': {'value': 1, 'dB': 0.0, 'unit': ['W', 'dBW']}, 'antennaGainTx': {'value': 2.0, 'dB': 3.010299956639812, 'unit': ['', 'dBi']}, 'eirp': {'value': 2, 'dB': 3.010299956639812, 'unit': ['W', 'dBW']}, 'distance': {'value': 804437, 'dB': '', 'unit': ['m', '']}, 'pathloss': {'value': 24186303188241.7, 'dB': 133.83569492684344, 'unit': ['', 'dB']}, 'powerFluxDensity': {'value': 2.4594390041272217e-13, 'dB': '', 'unit': ['W/m²', '']}, 'antennaDiameter': {'value': 6.709523178178463, 'dB': '', 'unit': ['m', '']}, 'antennaArea': {'value': 35.35681990451987, 'dB': '', 'unit': ['m²', '']}, 'antennaEffectiveArea': {'value': 21.21409194271192, 'dB': '', 'unit': ['m²', '']}, 'antennaEfficiency': {'value': 60.0, 'dB': '', 'unit': ['%', '']}, 'antennaGainRx': {'value': 63.09573444801933, 'd